In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the dataset and preprocess as before
df = pd.read_csv('/content/drive/MyDrive/HomeC.csv')

<ipython-input-3-3f25bf8cfb98>:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/HomeC.csv')


In [4]:
# Convert 'time' column to datetime format and coerce errors
df['time'] = pd.to_datetime(df['time'], errors='coerce')

In [5]:
# Define a single target column and weather-related input columns
target_column = ['House_overall[kW]']
weather_columns = ['temperature', 'humidity', 'visibility', 'apparentTemperature',
                   'pressure', 'windSpeed', 'windBearing', 'dewPoint']

In [6]:
# Drop rows with missing values in the target column and fill weather-related columns
df_cleaned = df.dropna(subset=target_column)
df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')

<ipython-input-6-1195c25159e3>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')


In [7]:
# Separate input features and target variable
X = df_cleaned[weather_columns]
y = df_cleaned[target_column]

In [8]:
# Scale features and target column
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [9]:
# Function to create sequences
def create_sequences(X, y, time_steps=30):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [10]:
# Generate sequences
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, time_steps=60)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42, shuffle=False)

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Dense, BatchNormalization, Softmax, MaxPooling1D, Bidirectional, LSTM, Input, Flatten,Dropout
from tensorflow.keras.models import Model

In [12]:
# Input shape (assuming X_train_seq is available)
input_shape = (X_train.shape[1], X_train.shape[2])  # Assuming X_train_seq has [samples, time_steps, features]

In [13]:
# Define the model
def build_model(input_shape):
    inputs = Input(shape=(input_shape[0], input_shape[1]))

    # Convolutional Layer
    conv_layer = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)

    # Permutation layer (FCC)
    dense_layer = Dense(128, activation='relu')(Flatten()(conv_layer))

    # Batch Normalization
    bn_layer = BatchNormalization()(dense_layer)

    # Softmax Activation
    softmax_layer = Softmax()(bn_layer)

    # Reshape softmax output to 3D for MaxPooling using a Lambda layer
    softmax_reshaped = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(softmax_layer)

    # MaxPooling
    max_pooling = MaxPooling1D(pool_size=2, strides=2)(softmax_reshaped)

    lstm_layer = Bidirectional(LSTM(64, return_sequences=False))(max_pooling) # Assign output to lstm_layer
    x = Dropout(0.2)(lstm_layer) # Apply dropout to LSTM output
    x = Dense(64, activation='relu')(x)


    # Final Dense layer (output)
    outputs = Dense(1, activation='linear')(x)  # Adjust activation based on regression or classification, Use output from previous layer

    # Build Model
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [14]:
# Build the model
model = build_model(input_shape)



In [15]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_absolute_error', metrics=['mae'])


In [16]:
# Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 60, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 60, 64)              │           1,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3840)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         491,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Softmax)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 128, 1)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 64, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          33,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 535,873 (2.04 MB)

 Trainable params: 535,617 (2.04 MB)

 Non-trainable params: 256 (1.00 KB)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping


In [55]:
# Define EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    min_delta=0.001,   # Minimum change in the monitored quantity to qualify as an improvement
    patience=5,         # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model with EarlyStopping
history = model.fit(
    X_train, y_train,  # Assuming X_train and y_train are your training data
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]  # Add the EarlyStopping callback
)

Epoch 1/10
8818/8818 ━━━━━━━━━━━━━━━━━━━━ 687s 77ms/step - loss: 0.0276 - mae: 0.0276 - val_loss: 0.0728 - val_mae: 0.0728
Epoch 2/10
8818/8818 ━━━━━━━━━━━━━━━━━━━━ 718s 81ms/step - loss: 0.0268 - mae: 0.0268 - val_loss: 0.0718 - val_mae: 0.0718
Epoch 3/10
8818/8818 ━━━━━━━━━━━━━━━━━━━━ 734s 80ms/step - loss: 0.0267 - mae: 0.0267 - val_loss: 0.0731 - val_mae: 0.0731
Epoch 4/10
8818/8818 ━━━━━━━━━━━━━━━━━━━━ 727s 79ms/step - loss: 0.0263 - mae: 0.0263 - val_loss: 0.0723 - val_mae: 0.0723
Epoch 5/10
8818/8818 ━━━━━━━━━━━━━━━━━━━━ 695s 79ms/step - loss: 0.0262 - mae: 0.0262 - val_loss: 0.0738 - val_mae: 0.0738
Epoch 6/10
8818/8818 ━━━━━━━━━━━━━━━━━━━━ 697s 79ms/step - loss: 0.0260 - mae: 0.0260 - val_loss: 0.0722 - val_mae: 0.0722
Epoch 7/10
8818/8818 ━━━━━━━━━━━━━━━━━━━━ 704s 80ms/step - loss: 0.0258 - mae: 0.0258 - val_loss: 0.0728 - val_mae: 0.0728
